In [1]:
def quitar_acentos(s):
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
    )
    for a, b in replacements:
        s = s.replace(a, b).replace(a.upper(), b.upper())
    return s

def reset_name_column(df):
    '''
    reset_name_column: esta funcion da formato a los nombres de las columnas. Convierte los nombres a minusculas,
    reemplaza espacio por guión bajo y elimina caracteres especiales.

    arg:
        df: dataframe a renombrar
    return:
        df: dataframe renombrado.
    '''
    cols = df.columns
    cols_new = []
    for i in cols:
        i.lower()\
            .replace(" ", "_")\
            .replace("(", "")\
            .replace(")", "")
        cols_new.append(i)
    df.columns = cols_new
    return df 


In [2]:
# import utilitys # error con las funciones auxiliares.
import pandas as pd
import glob, os, zipfile
import numpy as np

# Accidentes automovilisticos

## Descargamos los datos.
Se utilizan los datos oficiales de la página de INEGI.

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /tmp/horse-or-human.zip

In [3]:
# descomprimos los datos del zip
local_zip = "../data/atus_anual_csv.zip"
zip_ref = zipfile.ZipFile(local_zip, "r")
zip_ref.extractall("../data/raw/accidentes/")

## Concatenamos los datos y guardamos un solo archivo

In [5]:
# juntamos todos los accidentes en uno solo archivo.
# get data file names
path =r'../data/raw/accidentes/conjunto_de_datos/'
filenames = glob.glob(path + "/*.csv")
dfs = []

# columnas que nos interesan. 
col_analisis=['id_entidad', 'id_municipio', 'anio', 'mes', 'id_hora',
       'id_minuto', 'id_dia', 'diasemana', 'tipaccid',
        'causaacci', 'caparod', 'sexo', 'aliento',
       'cinturon', 'id_edad', "clasacc"]

for filename in filenames:
    df_year=pd.read_csv(filename, index_col=False)
    df_year.columns = [item.lower() for item in df_year.columns.to_list()]
    dfs.append(df_year[col_analisis])
    print(filename)


# Concatenate all data into one DataFrame
big_frame = pd.concat(dfs, ignore_index=True)
big_frame.to_csv("../data/raw/accidentes/accidentes_1997_2019.csv", index=False)

../data/raw/accidentes/conjunto_de_datos/atus_anual_1997.csv
../data/raw/accidentes/conjunto_de_datos/atus_anual_2005.csv
../data/raw/accidentes/conjunto_de_datos/atus_anual_2000.csv
../data/raw/accidentes/conjunto_de_datos/atus_anual_1998.csv
../data/raw/accidentes/conjunto_de_datos/atus_anual_2002.csv
../data/raw/accidentes/conjunto_de_datos/atus_anual_2003.csv
../data/raw/accidentes/conjunto_de_datos/atus_anual_2019.csv
../data/raw/accidentes/conjunto_de_datos/atus_anual_2007.csv
../data/raw/accidentes/conjunto_de_datos/atus_anual_2013.csv
../data/raw/accidentes/conjunto_de_datos/atus_anual_2017.csv
../data/raw/accidentes/conjunto_de_datos/atus_anual_2016.csv
../data/raw/accidentes/conjunto_de_datos/atus_anual_2006.csv
../data/raw/accidentes/conjunto_de_datos/atus_anual_2012.csv
../data/raw/accidentes/conjunto_de_datos/atus_anual_1999.csv
../data/raw/accidentes/conjunto_de_datos/atus_anual_2008.csv
../data/raw/accidentes/conjunto_de_datos/atus_anual_2001.csv
../data/raw/accidentes/c

## Renombramos columnas, tratamos los valores nulos y transformamos el tipo de las columnas 

In [2]:
accidentes_mexico = pd.read_csv("../data/raw/accidentes/accidentes_1997_2019.csv")

In [3]:
accidentes_mexico.shape

(8876380, 16)

In [4]:
accidentes_mexico.head()

,id_entidad,id_municipio,anio,mes,id_hora,id_minuto,id_dia,diasemana,tipaccid,causaacci,caparod,sexo,aliento,cinturon,id_edad,clasacc
0,1,1.0,1997,1,20,0,7,Martes,Colisión con vehículo automotor,Conductor,Pavimentada,Hombre,Se ignora,Se ignora,18,Sólo daños
1,1,1.0,1997,1,20,0,19,Domingo,Colisión con vehículo automotor,Conductor,Pavimentada,Hombre,No,Se ignora,19,No fatal
2,1,1.0,1997,1,20,0,20,Lunes,Colisión con vehículo automotor,Conductor,Pavimentada,Hombre,No,Se ignora,43,Sólo daños
3,1,1.0,1997,1,20,5,22,Miércoles,Colisión con vehículo automotor,Falla del vehículo,Pavimentada,Hombre,No,Se ignora,26,No fatal
4,1,1.0,1997,1,20,15,25,Sábado,Colisión con vehículo automotor,Conductor,Pavimentada,Hombre,No,Se ignora,28,Sólo daños


In [5]:
accidentes_mexico.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8876380 entries, 0 to 8876379
Data columns (total 16 columns):
 #   Column        Dtype  
---  ------        -----  
 0   id_entidad    int64  
 1   id_municipio  float64
 2   anio          int64  
 3   mes           int64  
 4   id_hora       int64  
 5   id_minuto     int64  
 6   id_dia        int64  
 7   diasemana     object 
 8   tipaccid      object 
 9   causaacci     object 
 10  caparod       object 
 11  sexo          object 
 12  aliento       object 
 13  cinturon      object 
 14  id_edad       int64  
 15  clasacc       object 
dtypes: float64(1), int64(7), object(8)
memory usage: 1.1+ GB


### Agruegamos las entidades y municipios de los catalogos

In [8]:
print(accidentes_mexico.id_entidad.unique())

# concatenamos con los catalogos de entidad
cat_estado = pd.read_csv("../data/raw/accidentes/catalogos/tc_entidad.csv", index_col=False)
cat_estado.columns = [item.lower() for item in cat_estado.columns.to_list()]
cat_estado["nom_entidad"] = cat_estado["nom_entidad"].apply(lambda x: quitar_acentos(x)) 

print(accidentes_mexico.shape)
accidentes_mexico=pd.merge(accidentes_mexico, cat_estado, on="id_entidad", how="left")
print(accidentes_mexico.shape)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32]
(8876380, 16)
(8876380, 17)


In [9]:
accidentes_mexico.head()

,id_entidad,id_municipio,anio,mes,id_hora,id_minuto,id_dia,diasemana,tipaccid,causaacci,caparod,sexo,aliento,cinturon,id_edad,clasacc,nom_entidad
0,1,1.0,1997,1,20,0,7,Martes,Colisión con vehículo automotor,Conductor,Pavimentada,Hombre,Se ignora,Se ignora,18,Sólo daños,Aguascalientes
1,1,1.0,1997,1,20,0,19,Domingo,Colisión con vehículo automotor,Conductor,Pavimentada,Hombre,No,Se ignora,19,No fatal,Aguascalientes
2,1,1.0,1997,1,20,0,20,Lunes,Colisión con vehículo automotor,Conductor,Pavimentada,Hombre,No,Se ignora,43,Sólo daños,Aguascalientes
3,1,1.0,1997,1,20,5,22,Miércoles,Colisión con vehículo automotor,Falla del vehículo,Pavimentada,Hombre,No,Se ignora,26,No fatal,Aguascalientes
4,1,1.0,1997,1,20,15,25,Sábado,Colisión con vehículo automotor,Conductor,Pavimentada,Hombre,No,Se ignora,28,Sólo daños,Aguascalientes


In [11]:
print(len(accidentes_mexico.id_municipio.unique()))

# concatenamos con los catalogos de municipio
cat_muni=pd.read_csv("../data/raw/accidentes/catalogos/tc_municipio.csv", index_col=False)
cat_muni.columns = [item.lower() for item in cat_muni.columns.to_list()]

print(accidentes_mexico.shape)
accidentes_mexico=pd.merge(accidentes_mexico, cat_muni, on=["id_entidad","id_municipio"], how="left")
print(accidentes_mexico.shape)

571
(8876380, 17)
(8876380, 18)


In [13]:
accidentes_mexico.head()

,id_entidad,id_municipio,anio,mes,id_hora,id_minuto,id_dia,diasemana,tipaccid,causaacci,caparod,sexo,aliento,cinturon,id_edad,clasacc,nom_entidad,nom_municipio
0,1,1.0,1997,1,20,0,7,Martes,Colisión con vehículo automotor,Conductor,Pavimentada,Hombre,Se ignora,Se ignora,18,Sólo daños,Aguascalientes,Aguascalientes
1,1,1.0,1997,1,20,0,19,Domingo,Colisión con vehículo automotor,Conductor,Pavimentada,Hombre,No,Se ignora,19,No fatal,Aguascalientes,Aguascalientes
2,1,1.0,1997,1,20,0,20,Lunes,Colisión con vehículo automotor,Conductor,Pavimentada,Hombre,No,Se ignora,43,Sólo daños,Aguascalientes,Aguascalientes
3,1,1.0,1997,1,20,5,22,Miércoles,Colisión con vehículo automotor,Falla del vehículo,Pavimentada,Hombre,No,Se ignora,26,No fatal,Aguascalientes,Aguascalientes
4,1,1.0,1997,1,20,15,25,Sábado,Colisión con vehículo automotor,Conductor,Pavimentada,Hombre,No,Se ignora,28,Sólo daños,Aguascalientes,Aguascalientes


### Valores nulos en las columnas: id_dia, año, mes, diasemana, 

In [14]:
# imprimimos los años
accidentes_mexico.anio.unique()

array([1997, 2005, 2000, 1998, 2002, 2003, 2019, 2007, 2013, 2017, 2016,
       2006, 2012, 1999, 2008, 2001, 2009, 2014, 2010, 2015, 2018, 2004,
       2011])

In [15]:
accidentes_mexico.mes.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [16]:
accidentes_mexico.id_dia.unique()

array([ 7, 19, 20, 22, 25, 14, 17, 21, 12,  8,  6, 27, 15, 18,  5, 16, 29,
       11,  9,  3, 13, 10, 30, 24, 26,  2, 28, 23,  1,  4, 31, 32,  0])

#### Analicemos la existencia de ceros en los días


In [17]:
accidentes_dia_cero = accidentes_mexico[accidentes_mexico.id_dia==0]

In [20]:
accidentes_dia_cero.shape

(15210, 18)

In [18]:
accidentes_dia_cero.anio.unique() # todos estan en el año 2018

array([2018])

Observamos que los valores nulos provienen del año 2018.

In [17]:
accidentes_dia_cero.nom_entidad.value_counts()

Oaxaca                             6361
Puebla                             2207
Veracruz de Ignacio de la Llave    1584
Chiapas                            1128
Hidalgo                             865
San Luis Potosi                     434
Sonora                              431
Tlaxcala                            348
Tamaulipas                          265
Michoacan de Ocampo                 251
Yucatan                             227
Mexico                              211
Zacatecas                           164
Nuevo Leon                          164
Chihuahua                           150
Coahuila de Zaragoza                137
Nayarit                              73
Durango                              59
Guanajuato                           55
Tabasco                              25
Queretaro                            21
Aguascalientes                       19
Campeche                             11
Morelos                              11
Sinaloa                               7


 La mayoria estan en Oaxaca.

In [22]:
for i in accidentes_dia_cero.columns[4:16]:
    print("La columna",i,"tiene los valores únicos:", accidentes_dia_cero[i].unique())

La columna id_hora tiene los valores únicos: [0]
La columna id_minuto tiene los valores únicos: [0]
La columna id_dia tiene los valores únicos: [0]
La columna diasemana tiene los valores únicos: ['Certificado cero']
La columna tipaccid tiene los valores únicos: ['Certificado cero']
La columna causaacci tiene los valores únicos: ['Certificado cero']
La columna caparod tiene los valores únicos: ['Certificado cero']
La columna sexo tiene los valores únicos: ['Certificado cero']
La columna aliento tiene los valores únicos: ['Certificado cero']
La columna cinturon tiene los valores únicos: ['Certificado cero']
La columna id_edad tiene los valores únicos: [0]
La columna clasacc tiene los valores únicos: ['Certificado cero']


**Observemos que no existen valores nulos en todas las demás columnas**, por lo que filtraremos estos datos.

In [24]:
# Filtramos los que tenga en día 0.
accidentes_mexico = accidentes_mexico[accidentes_mexico.id_dia!=0]

# comprobamos que ya no esten 
accidentes_mexico.id_dia.unique()

array([ 7, 19, 20, 22, 25, 14, 17, 21, 12,  8,  6, 27, 15, 18,  5, 16, 29,
       11,  9,  3, 13, 10, 30, 24, 26,  2, 28, 23,  1,  4, 31, 32])

In [25]:
accidentes_mexico.diasemana.unique()

array(['Martes', 'Domingo', 'Lunes', 'Miércoles', 'Sábado', 'Viernes',
       'Jueves', 'Miercoles', 'Sabado', 'lunes', 'No especificado'],
      dtype=object)

In [26]:
accidentes_mexico.id_edad.unique()

array([18, 19, 43, 26, 28, 22,  0, 25, 30, 36, 21, 53, 34, 24, 35, 33, 56,
       31, 38, 62, 20, 47, 80, 42, 48, 46, 40, 32, 44, 39, 13, 23, 78, 29,
       41, 63, 27, 37, 45, 16, 76, 49, 55, 50, 52, 17, 14, 65, 58, 72, 64,
       60, 59, 67, 54, 57, 51, 93, 15, 69, 61, 66, 70, 77, 71, 68, 89, 73,
       74, 95, 75, 87, 12, 85, 84, 82, 79, 86, 83, 88, 81, 91, 99, 92, 90,
       98, 94, 97, 96])

In [28]:
accidentes_mexico[accidentes_mexico.id_edad==0].shape

(951135, 18)

In [29]:
accidentes_mexico["id_edad"]=accidentes_mexico["id_edad"].replace({0:np.nan,
                                                                  99:np.nan})

In [30]:
accidentes_mexico.head()

,id_entidad,id_municipio,anio,mes,id_hora,id_minuto,id_dia,diasemana,tipaccid,causaacci,caparod,sexo,aliento,cinturon,id_edad,clasacc,nom_entidad,nom_municipio
0,1,1.0,1997,1,20,0,7,Martes,Colisión con vehículo automotor,Conductor,Pavimentada,Hombre,Se ignora,Se ignora,18.0,Sólo daños,Aguascalientes,Aguascalientes
1,1,1.0,1997,1,20,0,19,Domingo,Colisión con vehículo automotor,Conductor,Pavimentada,Hombre,No,Se ignora,19.0,No fatal,Aguascalientes,Aguascalientes
2,1,1.0,1997,1,20,0,20,Lunes,Colisión con vehículo automotor,Conductor,Pavimentada,Hombre,No,Se ignora,43.0,Sólo daños,Aguascalientes,Aguascalientes
3,1,1.0,1997,1,20,5,22,Miércoles,Colisión con vehículo automotor,Falla del vehículo,Pavimentada,Hombre,No,Se ignora,26.0,No fatal,Aguascalientes,Aguascalientes
4,1,1.0,1997,1,20,15,25,Sábado,Colisión con vehículo automotor,Conductor,Pavimentada,Hombre,No,Se ignora,28.0,Sólo daños,Aguascalientes,Aguascalientes


### Transformamos el tipo de las columnas: anio, mes, diasemana, tipaccid, cuasacci, aliento.

In [31]:
for i in ["anio", "mes"]:
    accidentes_mexico[i]=accidentes_mexico[i].astype("str")

accidentes_mexico["id_dia"]=accidentes_mexico["id_dia"].astype(int).astype("str")

In [32]:
acentos = ["diasemana", "tipaccid", "causaacci", "aliento"]
for i in acentos:
    accidentes_mexico[i] = accidentes_mexico[i].apply(lambda x: quitar_acentos(x)) 


In [34]:
!mkdir ../data/processed/accidentes

In [35]:
accidentes_mexico.to_csv("../data/processed/accidentes/accidentes_1997_2019.csv", index=False)

# Registrados en circulación

In [3]:
import pandas as pd
import glob
import numpy as np

## Descargamos y descromprimos los datos

In [6]:
!mkdir ../data/raw/../data/raw/registros_automoviles

!wget --no-check-certificate \
    https://www.inegi.org.mx/contenidos/programas/vehiculosmotor/datosabiertos/vmrc_anual_csv.zip \
    -O ../data/raw/registros_automoviles/vmrc_anual_csv.zip

!wget --no-check-certificate \
    https://www.inegi.org.mx/contenidos/programas/vehiculosmotor/datosabiertos/vmrc_mensual_csv.zip
    -O ../data/raw/registros_automoviles/vmrc_mensual_csv.zip

mkdir: no se puede crear el directorio «../data/raw/../data/raw/registros_automoviles»: No existe el archivo o el directorio
--2021-07-19 13:07:02--  https://www.inegi.org.mx/contenidos/programas/vehiculosmotor/datosabiertos/vmrc_anual_csv.zip
Resolviendo www.inegi.org.mx (www.inegi.org.mx)... 200.23.8.5
Conectando con www.inegi.org.mx (www.inegi.org.mx)[200.23.8.5]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 1027126 (1003K) [application/x-zip-compressed]
Guardando como: “../data/raw/registros_automoviles/vmrc_anual_csv.zip”

../data/raw/registr 100%[===================>]   1003K  35.4KB/s    en 43s     

2021-07-19 13:07:47 (23.3 KB/s) - “../data/raw/registros_automoviles/vmrc_anual_csv.zip” guardado [1027126/1027126]



In [7]:
# descomprimos los datos del zip
local_zip = "../data/raw/registros_automoviles/vmrc_anual_csv.zip"
zip_ref = zipfile.ZipFile(local_zip, "r")
zip_ref.extractall("../data/raw/registros_automoviles/vmrc_anual/")

local_zip = "../data/raw/registros_automoviles/vmrc_mensual_csv.zip"
zip_ref = zipfile.ZipFile(local_zip, "r")
zip_ref.extractall("../data/raw/registros_automoviles/vmrc_mensual/")

## Conocimiento de la estructura de todos los archivos

In [8]:
vehiculos = pd.read_csv("../data/raw/registros_automoviles/vmrc_anual/conjunto_de_datos/vmrc_1997.csv", index_col=False)
vehiculos.columns = [item.lower() for item in vehiculos.columns.to_list()]
vehiculos.head(3)

,prod_est,cobertura,anio,id_entidad,id_municipio,auto_oficial,auto_publico,auto_particular,cam_pas_oficial,cam_pas_publico,cam_pas_particular,cyc_carga_oficial,cyc_carga_publico,cyc_carga_particular,moto_oficial,moto_de_alquiler,moto_particular,estatus
0,Vehículos de motor registrados en circulación....,Municipal,1997,1,1,1233,2212,81265,20,725,316,906,277,37905,146,0,6231,Cifras Definitivas.
1,Vehículos de motor registrados en circulación....,Municipal,1997,1,2,3,8,529,0,13,1,6,0,1589,0,0,13,Cifras Definitivas.
2,Vehículos de motor registrados en circulación....,Municipal,1997,1,3,9,63,2155,0,40,6,20,0,5124,2,0,69,Cifras Definitivas.


In [9]:
vehiculos.shape

(1495, 18)

In [10]:
vehiculos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1495 entries, 0 to 1494
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   prod_est              1495 non-null   object
 1   cobertura             1495 non-null   object
 2   anio                  1495 non-null   int64 
 3   id_entidad            1495 non-null   int64 
 4   id_municipio          1495 non-null   int64 
 5   auto_oficial          1495 non-null   int64 
 6   auto_publico          1495 non-null   int64 
 7   auto_particular       1495 non-null   int64 
 8   cam_pas_oficial       1495 non-null   int64 
 9   cam_pas_publico       1495 non-null   int64 
 10  cam_pas_particular    1495 non-null   int64 
 11  cyc_carga_oficial     1495 non-null   int64 
 12  cyc_carga_publico     1495 non-null   int64 
 13  cyc_carga_particular  1495 non-null   int64 
 14  moto_oficial          1495 non-null   int64 
 15  moto_de_alquiler      1495 non-null   

In [11]:
for i in ['prod_est', 'cobertura', 'anio', 'id_entidad', 'id_municipio']:
    print("Columna:" , i, "valores unicos", vehiculos[i].unique())

Columna: prod_est valores unicos ['Vehículos de motor registrados en circulación. Anual']
Columna: cobertura valores unicos ['Municipal']
Columna: anio valores unicos [1997]
Columna: id_entidad valores unicos [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32]
Columna: id_municipio valores unicos [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  40  57  59  61  65  68  74  78  81  89  97 101 106 108 109  39
  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  58  60
  62  63  64  66  67  73  75  69  70  71  72  76  77  79  80  82  83  84
  85  86  87  88  90  91  92  93  94  95  96  98  99 100 102 103 104 105
 107 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124   0 132
 140 153 154 156 164 169 172 174 186 197 207 208 125 126 127 128 129 130
 131 133 134 135 136 137 138 139 141 142 143 144 145 146 147 148 149 15

In [12]:
vehiculos = vehiculos.groupby(["anio", "id_entidad","id_municipio"], as_index=False)[vehiculos.columns[5:]].sum()

Solo se necesitaran las columnas: "anio", "id_entidad", "id_municipio", 'auto_oficial', 'auto_publico', 'auto_particular', 'cam_pas_oficial', 'cam_pas_publico', 'cam_pas_particular', 'cyc_carga_oficial', 'cyc_carga_publico', 'cyc_carga_particular', 'moto_oficial','moto_de_alquiler', 'moto_particular'.

## Concatenamos todos los archivos en uno solo.

In [15]:
# juntamos todos los accidentes en uno solo archivo.
# get data file names
path = '../data/raw/registros_automoviles/vmrc_anual/conjunto_de_datos/'
filenames = os.listdir(path)
dfs = []
# datos que nos interesan. 
col_analisis=["anio", "id_entidad", "id_municipio", 'auto_oficial', 'auto_publico', 'auto_particular', 
              'cam_pas_oficial', 'cam_pas_publico', 'cam_pas_particular', 'cyc_carga_oficial', 
              'cyc_carga_publico', 'cyc_carga_particular', 'moto_oficial','moto_de_alquiler', 
              'moto_particular']

for filename in filenames:
    df_year=pd.read_csv(path+filename, index_col=False)
    df_year.columns = [item.lower() for item in df_year.columns.to_list()]
    dfs.append(df_year[col_analisis])
    print(filename)

# Concatenate all data into one DataFrame
vehiculos_datos_completos = pd.concat(dfs, ignore_index=True)

vmrc_1984.csv
vmrc_1998.csv
vmrc_1987.csv
vmrc_2005.csv
vmrc_2008.csv
vmrc_1989.csv
vmrc_2018.csv
vmrc_2013.csv
vmrc_2012.csv
vmrc_2007.csv
vmrc_2009.csv
vmrc_2002.csv
vmrc_2017.csv
vmrc_1986.csv
vmrc_1997.csv
vmrc_1985.csv
vmrc_2010.csv
vmrc_1983.csv
vmrc_1980.csv
vmrc_2004.csv
vmrc_2015.csv
vmrc_1981.csv
vmrc_1999.csv
vmrc_1992.csv
vmrc_2016.csv
vmrc_1991.csv
vmrc_2000.csv
vmrc_1996.csv
vmrc_1990.csv
vmrc_1982.csv
vmrc_2006.csv
vmrc_1994.csv
vmrc_1988.csv
vmrc_2003.csv
vmrc_2019.csv
vmrc_2011.csv
vmrc_2014.csv
vmrc_1993.csv
vmrc_2001.csv
vmrc_1995.csv


In [16]:
for i in ['anio', 'id_entidad', 'id_municipio']:
    print("Columna:" , i, "valores unicos", vehiculos_datos_completos[i].unique())

Columna: anio valores unicos [1984 1998 1987 2005 2008 1989 2018 2013 2012 2007 2009 2002 2017 1986
 1997 1985 2010 1983 1980 2004 2015 1981 1999 1992 2016 1991 2000 1996
 1990 1982 2006 1994 1988 2003 2019 2011 2014 1993 2001 1995]
Columna: id_entidad valores unicos [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32]
Columna: id_municipio valores unicos [  1   2   3   4   5   6   7   8   9  10  17  18  20  25  28  30  32  33
  35 999  19  27  40  51  57  61  65  68  69  74  89  97 101 102 106 108
 109  11  21  24  31  36  37  45  48  50  52  55  60  62  12  13  14  15
  16  38  23  42  46  29  34  53  39  41  56  63  67  70  76  77  84  22
  43  58  73  78  82  83  85  93  94 100 113 114 118 121 124  54  81  88
  92  95  99 104 110 120  66  71  75  86  98 107 112  26  44  49   0 119
 132 140 153 154 156 164 174 186 194 197 207 208  87 115 123 126 128 131
 133 138 141 148 152 155 160 161 175 181 183 189 192 193 196  59  96  47
  64  72  79

In [17]:
vehiculos_datos_completos.head()

,anio,id_entidad,id_municipio,auto_oficial,auto_publico,auto_particular,cam_pas_oficial,cam_pas_publico,cam_pas_particular,cyc_carga_oficial,cyc_carga_publico,cyc_carga_particular,moto_oficial,moto_de_alquiler,moto_particular
0,1984,1,1,1105,747,30263,33,516,71,948,590,18231,34,0,2032
1,1984,1,2,2,23,123,1,0,2,2,0,395,0,0,49
2,1984,1,3,4,44,399,0,0,1,12,1,1155,0,0,45
3,1984,1,4,1,9,91,0,0,0,4,0,154,0,0,8
4,1984,1,5,3,11,262,0,11,1,3,0,560,0,0,76


In [18]:
vehiculos_datos_completos[vehiculos_datos_completos.id_municipio.isin([0,999])]

,anio,id_entidad,id_municipio,auto_oficial,auto_publico,auto_particular,cam_pas_oficial,cam_pas_publico,cam_pas_particular,cyc_carga_oficial,cyc_carga_publico,cyc_carga_particular,moto_oficial,moto_de_alquiler,moto_particular
37,1984,5,999,25,314,13550,5,185,55,12,192,9333,4,0,463
67,1984,7,999,5,42,1282,0,75,10,5,112,2167,0,0,15
90,1984,8,999,35,135,2857,9,49,10,50,190,7743,2,0,204
116,1984,10,999,8,181,1777,18,113,11,73,239,4769,0,0,322
136,1984,11,999,0,361,7936,0,194,20,11,106,11952,4,0,1052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57359,1995,26,999,0,0,691,0,1,0,0,2,105,0,0,1
57401,1995,28,999,0,4,2083,0,0,0,0,0,5710,0,0,2
57477,1995,30,999,0,1140,13089,0,347,1004,0,224,20860,0,0,276
57496,1995,31,999,0,443,4095,0,5,0,0,96,3890,0,0,0


In [19]:
vehiculos_datos_completos[vehiculos_datos_completos.id_municipio.isin([0,999])].id_entidad.unique()

array([ 5,  7,  8, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 24, 26, 28,
       29, 30, 31, 32, 18, 25, 27,  9, 23])

In [22]:
# concatenamos con los catalogos
cat_estado = pd.read_csv("../data/raw/registros_automoviles/vmrc_anual/catalogos/tc_entidad.csv", index_col=False)
cat_estado.columns = [item.lower() for item in cat_estado.columns.to_list()]
cat_estado["nom_entidad"] = cat_estado["nom_entidad"].apply(lambda x: quitar_acentos(x)) 

print(vehiculos_datos_completos.shape)
vehiculos_datos_completos=pd.merge(vehiculos_datos_completos, cat_estado, on="id_entidad", how="left")
print(vehiculos_datos_completos.shape)

(57538, 15)
(57538, 16)


In [23]:
cat_municipio = pd.read_csv("../data/raw/registros_automoviles/vmrc_anual/catalogos/tc_municipio.csv", index_col=False)
cat_municipio.columns = [item.lower() for item in cat_municipio.columns.to_list()]
cat_estado["nom_entidad"] = cat_estado["nom_entidad"].apply(lambda x: quitar_acentos(x)) 

print(vehiculos_datos_completos.shape)
vehiculos_datos_completos=pd.merge(vehiculos_datos_completos, cat_municipio, on=["id_entidad", "id_municipio"],
                                   how="left")
print(vehiculos_datos_completos.shape)

(57538, 16)
(57538, 17)


In [25]:
vehiculos_datos_completos.to_csv("../data/processed/registros_automaviles_anual_1997_2019.csv", index=False)

## Limpieza datos vehiculos mensuales

In [46]:
path = "../data/raw/registros_automoviles/vmrc_mensual/"
path_mensual = os.listdir(path)
filename = []
for i in path_mensual:
    filename.append(path+ i+"/conjunto_de_datos/"+os.listdir(path+i+"/conjunto_de_datos/")[0])

In [56]:
filename

['../data/raw/registros_automoviles/vmrc_mensual/vmrc_produccion_mensual_1983_2021/conjunto_de_datos/vmrc_produccion_mensual_tr_cifra_1983_2021.csv',
 '../data/raw/registros_automoviles/vmrc_mensual/vmrc_venta_veh_mensual_1988_2021/conjunto_de_datos/vmrc_venta_veh_mensual_tr_cifra_1988_2021.csv',
 '../data/raw/registros_automoviles/vmrc_mensual/vmrc_vehiculos_mensual_1991_2021/conjunto_de_datos/vmrc_vehiculos_mensual_tr_cifra_1991_2021.csv',
 '../data/raw/registros_automoviles/vmrc_mensual/vmrc_venta_cam_mensual_1995_2021/conjunto_de_datos/vmrc_venta_cam_mensual_tr_cifra_1995_2021.csv']

In [52]:
vehiculos_mensual = pd.read_csv(filename[3], index_col=False)
vehiculos_mensual.head()

,PROD_EST,COBERTURA,ANIO,ID_MES,USO_MULTIPLE_NACIONAL,USO_MULTIPLE_IMPORTADO,CLASE1_NACIONAL,CLASE1_IMPORTADO,CLASE2_NACIONAL,CLASE2_IMPORTADO,...,CLASE3_IMPORTADO,CLASE4_5_6_7_8_NACIONAL,CLASE4_5_6_7_8_IMPORTADO,TRACTOCAMION_NACIONAL,TRACTOCAMION_IMPORTADO,CHASÍS_CORAZA_NACIONAL,CHASÍS_CORAZA_IMPORTADO,AUTOBUS_INTEGRAL_NACIONAL,AUTOBUS_INTEGRAL_IMPORTADO,ESTATUS
0,Vehículos de motor registrados en circulación....,Nacional,1995,1,1778,1807,2093,241,3915,90,...,0,604,397.0,120,0.0,33,0.0,17,2.0,Cifras Definitivas.
1,Vehículos de motor registrados en circulación....,Nacional,1995,2,1839,836,1598,120,2872,47,...,0,372,204.0,60,16.0,36,0.0,24,15.0,Cifras Definitivas.
2,Vehículos de motor registrados en circulación....,Nacional,1995,3,1652,936,2288,144,2538,48,...,0,554,150.0,42,0.0,92,0.0,15,31.0,Cifras Definitivas.
3,Vehículos de motor registrados en circulación....,Nacional,1995,4,1205,378,1184,56,1306,11,...,0,158,35.0,31,15.0,9,0.0,5,1.0,Cifras Definitivas.
4,Vehículos de motor registrados en circulación....,Nacional,1995,5,843,434,1267,86,1503,48,...,0,282,47.0,38,0.0,31,0.0,3,1.0,Cifras Definitivas.


Existen datos de los vehiculos registrados por mes desde 1991 a 2018, pero no se encuentran a nivel municipal por lo que se considera que estos datos no son servirián para el analisis buscado. 

In [53]:
vehiculos_mensual.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318 entries, 0 to 317
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   PROD_EST                    318 non-null    object 
 1   COBERTURA                   318 non-null    object 
 2   ANIO                        318 non-null    int64  
 3   ID_MES                      318 non-null    int64  
 4   USO_MULTIPLE_NACIONAL       318 non-null    int64  
 5   USO_MULTIPLE_IMPORTADO      318 non-null    int64  
 6   CLASE1_NACIONAL             318 non-null    int64  
 7   CLASE1_IMPORTADO            318 non-null    int64  
 8   CLASE2_NACIONAL             318 non-null    int64  
 9   CLASE2_IMPORTADO            318 non-null    int64  
 10  CLASE3_NACIONAL             318 non-null    int64  
 11  CLASE3_IMPORTADO            318 non-null    int64  
 12  CLASE4_5_6_7_8_NACIONAL     318 non-null    int64  
 13  CLASE4_5_6_7_8_IMPORTADO    201 non

In [55]:
vehiculos_mensual.ANIO.unique()

array([1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005,
       2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016,
       2017, 2018, 2019, 2020, 2021])